In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

from caen_felib import device

## Connect

In [ ]:
dig = device.connect('dig2://caendgtz-eth-16218.caen.local/')

## Example of node browsing

In [ ]:
par_name = 'RecordLengthS'
print(f'Info of parameter {par_name}:')

par_node = dig.par[par_name]

# Browse parameter child nodes
for child in dig.par[par_name]:
    print(f'\t{child.name} == {child.value}')

# There are several equivalent ways a node can be retrieved
assert(par_node == dig.get_node(f'/par/{par_name}'))
assert(par_node == dig.par.get_node(f'/{par_name}'))
assert(par_node == dig[f'par/{par_name}'])
assert(par_node == dig.par[par_name])
assert(par_node == dig.get_node('/par')[par_name])
assert(par_node == dig.par.RecordLengthS)
assert(par_node == par_node.child_nodes[0].parent_node)
assert(par_node == par_node.get_child_nodes()[0].get_parent_node())

## Configure digitizer

In [ ]:
# Reset
dig.cmd.reset()

# Set some digitizer parameters
dig.par.AcqTriggerSource.value = 'SwTrg'
dig.par.RecordLengthS.value = f'{4096}'
dig.par.PreTriggerS.value = f'{128}'

# Set some channel parameters
for ch in dig.ch:
	ch.par.DCOffset.value = f'{50}'

## Configure endpoint

In [ ]:
nch = int(dig.par.numch.value)
reclen = int(dig.par.recordlengths.value)

data_format = [
	{
		'name': 'TIMESTAMP',
		'type': 'U64',
	},
	{
		'name': 'WAVEFORM',
		'type': 'U16',
		'dim': 2,
		'shape': [nch, reclen],
	},
	{
		'name': 'WAVEFORM_SIZE',
		'type': 'U64',
		'dim': 1,
		'shape': [nch],
	},
]

# set_read_data_format returns allocated buffers
data = dig.endpoint.scope.set_read_data_format(data_format)
timestamp = data[0].value
waveform = data[1].value
waveform_size = data[2].value

# set enabled endpoint to activate decode
dig.endpoint.par.ActiveEndpoint.value = 'scope'


## Test waveform plot

In [ ]:
# Start acquisition
dig.cmd.armacquisition()
dig.cmd.swstartacquisition()

# Send trigger and wait for first event
dig.cmd.sendswtrigger()
dig.endpoint.scope.read_data(-1, data)

# Stop acquisition
dig.cmd.disarmacquisition()

# Plot waveforms
for i in range(nch):
    size = waveform_size[i]
    plt.plot(np.arange(size), waveform[i][:size])
plt.title(f'Timestamp: {timestamp}')
plt.show()

## Some statistics
Following lines assume that all channels are enabled and that all waveforms have the same size, and that the size is equal to the np.ndarray row size.

In [ ]:
for s, w in zip(waveform_size, waveform):
    if s != w.size:
        raise RuntimeError('Invalid waveform size')

# Compute statistics
waveform_mean = waveform.mean(1)
waveform_std = waveform.std(1)
corrcoef = np.corrcoef(waveform)

fig = plt.figure(figsize=(10, 5))

# Plot 1
ax1 = fig.add_subplot(121)

pl1 = ax1.scatter(waveform_mean, waveform_std)
for i in range(len(waveform)):
    ax1.annotate(f'{i}', (waveform_mean[i], waveform_std[i]))

ax1.set_xlabel('Mean (ADC counts)')
ax1.set_ylabel('Standard deviation (ADC counts)')
ax1.set_title('Channel waveform statistics')
ax1.set_aspect('auto')

# Plot 2
ax2 = fig.add_subplot(122)
pl2 = ax2.imshow(corrcoef, interpolation='nearest')

ax2.set_xlabel('Channel')
ax2.set_ylabel('Channel')
ax2.set_title('Correlation coefficient between channels')
ax2.set_aspect('auto')
fig.colorbar(pl2)

# Show
plt.show()